In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyxona import File
import quantities as pq
import ripple_detection as rd
from scipy.signal import butter, lfilter
import tools

from tqdm import tqdm

In [5]:
path = "/home/jovyan/work/RippleSpindleCoupling/data/*.set"

df_info = tools.generate_fileinfo(path)

In [4]:
df_info

,fname,id,date,session
0,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL190,200917,1
1,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL197,201107,1
2,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL196,201015,2
3,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL191,200921,2
4,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL190,200916,1
5,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL190,200921,1
6,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL191,200914,1
7,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL190,200922,1
8,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL191,200916,2
9,/home/jovyan/work/RippleSpindleCoupling/data/K...,KKL196,201020,2


>For ripple detection (Boyce et al., 2016; Nakashiba et al., 2009), the LFP obtained from CA1
pyramidal cell layer was first band-pass filtered (100–250 Hz), and amplitude was calculated using
the Hilbert transform. Ripple windows were characterized as signals that exceed the amplitude
threshold (three times the standard deviation). Signals that were less than 50 ms apart were merged.

> For spindle detection (Eschenko et al., 2006; Phillips et al., 2012), the LFP obtained from ACC
was band-pass filtered (12–15 Hz), and amplitude was calculated using the Hilbert transform. Spindle
windows were characterized as signals that exceed the amplitude threshold (two times the standard
deviation), with minimum and maximum duration of 200 and 2000 ms, respectively. Signals that are
less than 100 ms apart were merged.

In [ ]:
params_spindles = {
    'freq_min': 12,
    'freq_max': 15,
    'cheby1_ord': 4,
    'cheby1_ripple': 0.1,
    'dur_min': 0.15,
    'dur_max': 0.5,
    'dur_merge': 0.1,
    'ampl_thresh': 1.5,
    'return_info': 
        {'evts_ts_strtstop' : True,                                                                                                                                                                                                                                                                                                                                                                 
         'evts_ts_center' : True,                                                                                                                                                                                    
         'evts_ampl' : [-4, 4],
         'evts_data' : True}
}

In [ ]:

params_ripples = {
    'freq_min': 100,
    'freq_max': 250,
    'cheby1_ord': 4,
    'cheby1_ripple': 0.1,
    'dur_min': 0.01,
    'dur_max': 1.,
#    'dur_merge': 0.05,
    'dur_merge': False,
    'ampl_thresh': 3.,
    'return_info': 
        {'evts_ts_strtstop' : True,                                                                                                                                                                                                                                                                                                                                                                 
         'evts_ts_center' : True,                                                                                                                                                                                    
         'evts_ampl' : [-4, 4],
         'evts_data' : True}
}

In [ ]:
ls_df_rppls = []
ls_df_spndls = []

for index, row in df_fnames.iterrows():
    fname = row['animal']+'_'+str(row['date'])+'_'+str(row['session'])+'.set'
    f = File(path+fname)
    lfp_hpc = f.analog_signals[0]
    lfp_ctx = f.analog_signals[1]

    df_rppls_i = tools.event_detection(
        lfp_hpc.signal.magnitude,
        lfp_hpc.sample_rate.magnitude,
        **params_ripples)
        
    df_spndls_i = tools.event_detection(
        lfp_ctx.signal.magnitude,
        lfp_ctx.sample_rate.magnitude,
        **params_spindles)
    
    # merge with df_fnames
    d = row.to_dict()
    df_fnames_i = pd.DataFrame({k: [v] for k, v in d.items()})
    df_fnames_i['tmp'] = 1
    
    df_rppls_i['tmp'] = 1
    df_rppls_i = pd.merge(df_fnames_i, df_rppls_i, on=['tmp'])
    df_rppls_i.drop('tmp', axis=1, inplace=True)

    df_spndls_i['tmp'] = 1
    df_spndls_i = pd.merge(df_fnames_i, df_spndls_i, on=['tmp'])
    df_spndls_i.drop('tmp', axis=1, inplace=True)

    ls_df_rppls.append(df_rppls_i)
    ls_df_spndls.append(df_spndls_i)
    
df_rppls = pd.concat(ls_df_rppls, ignore_index=True, sort=False)
df_spndls = pd.concat(ls_df_spndls, ignore_index=True, sort=False)

In [ ]:
# TODO:# Visualize ripples, Spindles, make sure you are really computing the right thing.


### Event rate by animal

In [ ]:
# We take last ripple event as timepoint when recording stops. Imperfect, but okay for now.
t_trial = df_rppls.groupby(['animal', 'date', 'session'])['evts_ts_center'].max()
t_trial

In [ ]:
n_rppls_trial = df_rppls.groupby(['animal', 'date', 'session']).size()
n_rppls_trial

In [ ]:
n_spndls_trial = df_spndls.groupby(['animal', 'date', 'session']).size()
n_spndls_trial

In [ ]:
n_rppls_trial/t_trial

In [ ]:
n_spndls_trial/t_trial